In [1]:
import chatbot
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
from dotenv import load_dotenv
from openai import OpenAI
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import time

In [2]:
load_dotenv()
client = OpenAI()

index, metadata = chatbot.load_faiss_index(chatbot.VECTOR_DIR)
bm25=chatbot.build_bm25(metadata)
print("Index and metadata loaded.")

Index and metadata loaded.


In [3]:
with open("evaluation/evaluation_dataset_v2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [4]:
def json_to_documents(data: dict):
    documents = []

    for source, content in data.items():
        qa_list = content.get("perguntas_respostas", [])

        for item in qa_list:
            documents.append({
                "question": item.get("pergunta", ""),
                "answer": item.get("resposta", ""),
                "source": source
            })

    return documents


In [5]:
documents = json_to_documents(data)
documents

[{'question': 'O que é o Alentejo 2030?',
  'answer': 'O Alentejo 2030 é um programa que visa transformar vidas na região do Alentejo, impulsionando a inovação, a sustentabilidade e o crescimento regional, além de facilitar o acesso aos Fundos Europeus disponíveis.',
  'source': 'alentejo2030'},
 {'question': 'Quais são alguns dos eventos programados para o Alentejo 2030?',
  'answer': "Alguns dos eventos programados incluem um workshop sobre Prospetivas e Políticas Públicas em 15 de dezembro de 2025, uma conferência e espetáculo sobre Inteligência Artificial em 27 de novembro de 2025, e um concerto intitulado 'Ecoando a voz do Alentejo' em 16 de novembro de 2025.",
  'source': 'alentejo2030'},
 {'question': 'Qual foi a decisão da Comissão Europeia em relação ao Alentejo 2030?',
  'answer': 'A Comissão Europeia aprovou, em 15 de dezembro de 2025, a reprogramação do Programa Regional do Alentejo 2030, permitindo alinhar os fundos europeus com novas prioridades estratégicas e desafios em

In [6]:
# ---------------- Embedding Helper ----------------
def get_embedding(text: str):
    """
    Returns embedding using your OpenAI client
    """
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return np.array(response.data[0].embedding, dtype=np.float32).reshape(1, -1)

# ---------------- Core Metrics ----------------
def f1_score(pred, ref):
    if not pred: return 0.0
    pred_tokens = pred.split()
    ref_tokens = ref.split()
    common = Counter(pred_tokens) & Counter(ref_tokens)
    num_same = sum(common.values())
    if num_same == 0: return 0.0
    precision = num_same / len(pred_tokens)
    recall = num_same / len(ref_tokens)
    return 2 * precision * recall / (precision + recall)

def conciseness_score(generated, reference):
    return len(reference.split()) / max(1, len(generated.split()))

def semantic_similarity_score(generated, references):
    """
    Computes max semantic similarity between generated answer and reference answers
    """
    gen_emb = get_embedding(generated)
    ref_embs = np.vstack([get_embedding(ref) for ref in references])
    sims = cosine_similarity(gen_emb, ref_embs)[0]
    return float(np.max(sims))


# ---------------- Embedding-based Retrieval Success ----------------
def retrieval_success_semantic(retrieved_docs, reference):
    """
    Returns the maximum semantic similarity between the reference and retrieved docs.
    If no docs are retrieved, returns 0.0.
    """
    if not retrieved_docs:
        return 0.0

    ref_emb = get_embedding(reference)
    max_sim = 0.0

    for doc in retrieved_docs:
        doc_text = doc.get("content", "").strip()
        if not doc_text:
            continue
        doc_emb = get_embedding(doc_text)
        sim = cosine_similarity(ref_emb, doc_emb)[0][0]
        if sim > max_sim:
            max_sim = sim

    return float(max_sim)


# ---------------- Embedding-based Grounding ----------------
def grounding_score_semantic(generated, retrieved_docs):
    """
    Fraction of generated answer semantically supported by retrieved docs
    """
    if not retrieved_docs:
        return 0.0

    gen_emb = get_embedding(generated)
    doc_embs = np.vstack([get_embedding(doc.get("content","")) for doc in retrieved_docs])
    sims = cosine_similarity(gen_emb, doc_embs)[0]
    return float(np.max(sims)) if len(sims) > 0 else 0.0



In [7]:
def evaluate_rag_semantic(dataset, index, metadata, bm25, 
                          k, model="gpt-4o-mini", use_rerank = False):
    results = []

    for item in dataset:
        question = item["question"]
        reference = item["answer"]
        source = item.get("source", "unknown")

        # Generate answer
        start_time = time.time()
        generated_answer, retrieved_docs = chatbot.answer(question, index, 
                                                          metadata, bm25,
                                                          k, model)
        latency = time.time() - start_time

        if isinstance(generated_answer, tuple):
            generated_answer = generated_answer[0]

        # Metrics
        retrieval_success = retrieval_success_semantic(retrieved_docs, reference)
        grounding = grounding_score_semantic(generated_answer, retrieved_docs)
        f1 = f1_score(generated_answer, reference)
        conciseness = conciseness_score(generated_answer, reference)
        semantic_sim = semantic_similarity_score(generated_answer, [reference])

        results.append({
            "Question": question,
            "Reference Answer": reference,
            "Source": source,
            "Generated Answer": generated_answer,
            "Retrieval Success (semantic)": retrieval_success,
            "Grounding (semantic)": grounding,
            "F1": f1,
            "Conciseness": conciseness,
            "Semantic Similarity": semantic_sim,
            "Latency (s)": latency
        })

    df = pd.DataFrame(results)
    return df


In [8]:
results = {}
for k in [10]:
    results[k] = evaluate_rag_semantic(documents, index, metadata, bm25, k=k)



Assistente: O Alentejo 2030 é um programa regional aprovado pela Comissão Europeia para o período de 2021-2027, que visa promover a inovação, a sustentabilidade e o crescimento da região do Alentejo. Através deste programa, a região dispõe de uma dotação global de fundos europeus, com o objetivo de transformar vidas e contribuir para o desenvolvimento local. O Alentejo 2030 apoia diversas áreas e projetos que visam melhorar a qualidade de vida dos cidadãos e impulsionar a economia da região. 

Se precisar de mais informações ou ajuda sobre outro tema, estou à disposição!

Assistente: O documento menciona um evento que celebra projetos transformadores em Portugal, incluindo iniciativas do Alentejo 2030. Este evento é descrito como um ponto de encontro dinâmico, onde ocorrem debates, apresentações e momentos de partilha, além da cerimónia de entrega dos Prémios dos Fundos Europeus, que reconhecem projetos que estão a transformar a região.

No entanto, não são especificados outros evento

In [9]:
results[10].describe()

,Retrieval Success (semantic),Grounding (semantic),F1,Conciseness,Semantic Similarity,Latency (s)
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
mean,0.781298,0.813081,0.365315,0.473169,0.804216,2.139452
std,0.131120,0.056584,0.146581,0.176682,0.105822,0.486182
min,0.422468,0.710226,0.085470,0.223776,0.522829,1.459662
25%,0.699726,0.770324,0.260572,0.326811,0.771621,1.770565
50%,0.795816,0.817795,0.351145,0.455696,0.831597,2.064620
75%,0.888576,0.843168,0.454515,0.600672,0.880902,2.407853
max,0.979310,0.926311,0.683544,0.880000,0.934519,3.729602


In [10]:
results[10][results[10]['F1'] < 0.3]

,Question,Reference Answer,Source,Generated Answer,Retrieval Success (semantic),Grounding (semantic),F1,Conciseness,Semantic Similarity,Latency (s)
1,Quais são alguns dos eventos programados para ...,Alguns dos eventos programados incluem um work...,alentejo2030,O documento menciona um evento que celebra pro...,0.616848,0.841494,0.216667,0.621622,0.591383,2.106038
3,Quais são algumas das novas prioridades introd...,"As novas prioridades incluem a Defesa, a Água,...",alentejo2030,Algumas das novas prioridades introduzidas pel...,0.591603,0.828334,0.215686,0.500000,0.665305,2.224858
4,Como a reprogramação do Alentejo 2030 impacta ...,A reprogramação introduz maior flexibilidade n...,alentejo2030,A reprogramação do Alentejo 2030 impacta o apo...,0.660948,0.769996,0.205882,0.346535,0.642367,2.391629
6,Quais são algumas das iniciativas relacionadas...,As iniciativas incluem a redução de perdas nas...,algarve2030,Algumas das iniciativas relacionadas ao ciclo ...,0.846018,0.748796,0.289157,0.307087,0.759286,2.739043
9,Quais são as áreas de investimento previstas n...,As áreas de investimento incluem o aumento da ...,algarve2030,As áreas de investimento previstas no Programa...,0.585701,0.789700,0.265957,0.296552,0.666882,2.687753
13,Como o Programa Centro 2030 combate a fraude n...,O Programa Centro 2030 combate a fraude nos fu...,centro2030,O Programa Centro 2030 combate a fraude nos fu...,0.824841,0.813269,0.260870,0.286713,0.859217,2.894196
14,Quais são algumas das áreas de apoio incluídas...,As áreas de apoio incluídas na reprogramação d...,centro2030,Algumas das áreas de apoio incluídas na reprog...,0.643108,0.710226,0.271605,0.306452,0.769852,2.311711
16,Quais são os três pilares principais do progra...,Os três pilares principais do programa COMPETE...,compete2030,Os três pilares principais do programa COMPETE...,0.716496,0.735499,0.260274,0.489796,0.810025,2.875857
17,Como o programa COMPETE 2030 apoia as empresas?,O programa COMPETE 2030 apoia as empresas atra...,compete2030,O programa COMPETE 2030 apoia as empresas de v...,0.765818,0.776242,0.256098,0.223881,0.822518,2.513299
21,Quais são os principais serviços oferecidos pe...,Os principais serviços oferecidos pelo IAPMEI ...,iapmei,Os principais serviços oferecidos pelo IAPMEI ...,0.693468,0.770653,0.193548,0.250000,0.801644,2.424076
